# A Curious Course on Coroutines and Concurrency 

## Disclaimers

- All content of this jupyter notebook are referred to **David Beazley's [slide](http://117.128.6.29/cache/www.dabeaz.com/coroutines/Coroutines.pdf?ich_args2=526-03152400039188_616107d40934da0c48d67a98c79844af_10001002_9c896228dfc2f6d19f32518939a83798_460d1e2e54e6fd2148466980ccb734cb)**.


- I made some revisions in order to run correctly.


- Please run in Python3.

## Part 2 Coroutines, Pipelines, and Dataflow

### 2.1. Processing Pipelines

- Coroutines can be used to set up pipes

- You just chain coroutines together and push data through the pipe with send() operations

![img](img/processing_pipelines.png)

### 2.2. Pipelines Sources

- The pipline needs an initial source(a producer)

  ![img](img/pipline_sources.png)
  

- The source drives the entire pipeline

  ```
  def source(target):
      while not done:
          item = produce_an_item()
          ...
          target.send(item)
          ...
      target.close()        
  ```
  

- It is typically not a coroutine

### 2.3. Pipeline Sinks

- The pipline must have an end-point(sink)
  ![img](img/pipline_sinks.png)

- Collects all data sent to it and processes it

```
@coroutine
def sink():
    try:
        while True:
            item = (yield)  # Receive an item
            ...
     except GeneratorExit:
         # Done
         ...
```

### 2.3. An Example

- A picture
  ![img](img/a_example_source_sink.png)
  
- Critical point
  
  follow() is driving the entire computation by reading lines and pushing them into the printer() coroutine.

In [40]:
def coroutine(func):
    def start(*args, **kwargs):
        cor = func(*args, **kwargs)
        next(cor)
        return cor
    return start

In [39]:
import time

def follow(thefile, target):
    """ A source that mimics Unix 'tail -f' """
    # Go to the end of thefile
    thefile.seek(0)
    while True:
        line = thefile.readline()
        curr_pos = thefile.tell()
        print("curr_pos:", curr_pos)
        if not line:
            # time.sleep(0.1)
            # continue
            break
        target.send(line)
        
@coroutine
def printer():
    """ A sin that just prints the lines """
    try:
        while True:
            line = (yield)
            print(line)
    except GeneratorExit:
        print("Going away. Goodbye")


# Hooking it together
with open("access-log") as f:
    follow(f, printer())

curr_pos: 17
It is summer day

curr_pos: 51
I am studying Coroutine in Python

curr_pos: 76
Hi, ladies and gentlemen

curr_pos: 76
Going away. Goodbye


### 2.3. Pipeline Filters

- Intermediate stages both receive and send


- Tyically perform some kind of data transformation, filtering, routing, etc.

```
@coroutine
def filter(target):
    while True:
        item = (yield)  # Receive an item
        # Transform/filter item
        ...
        # Send it along to the next stage
        target.send(item)
```